In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/'My Drive'/'All_Repos'/Self-Driving-Car-NanoDegree/CarND-Behavioral-Cloning-P3

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Behavioral-Cloning-P3


## Loading and Displaying data

In [0]:
import matplotlib.pyplot as plt
IMAGE_HEIGHT= 80
IMAGE_WIDTH = 320
IMAGE_CHANNEL = 3
BATCH_SIZE = 128
EPOCHS = 5
# fuction to read image from file
def get_image(index, data):
    
    path = 'data/'+os.path.join(*data['center'].iloc[0].replace('\\', '/').split('/')[-3:])
    #print(path)
    angle = data.steer_angle.iloc[0]
    #print(angle)
    #if j == 'left':
    #    angle+=0.2
    #if j == 'right':
    #    angle-=0.2
    

    image = cv2.imread(path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    image = image[70:150,:,:]
    
    return image,angle

def generator(data):
    while True:
        # Randomize the indices to make an array
        #indices_arr = np.random.permutation(data.count()[0])
        # Not randomizing since the dataset is already shuffeled
        indices_arr = np.arange(data.count()[0])
        for batch in range(0, len(indices_arr), BATCH_SIZE):
            # slice out the current batch according to batch-size
            current_batch = indices_arr[batch:(batch + BATCH_SIZE)]

            # initializing the arrays, x_train and y_train
            X ,y = ([],[])

            for i in current_batch:
                # get an image and its corresponding color for an traffic light
                #for j in ['center','left','right']:
                #    [image, angle] = get_image(i, data, j)

                    # Appending them to existing batch
                #    X.append(image)
                #    y.append(angle)
                image, angle = get_image(i, data)
                X.append(image)
                y.append(angle)
            #print(y_train)

            yield np.array(X), np.array(y)


In [0]:
def get_model(time_len=1):

    model = Sequential()
    model.add(Lambda(lambda x: x/127. -1,input_shape=(80,320,3)))
    # Add three 5x5 convolution layers (output depth 24, 36, and 48), each with 2x2 stride
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())

    #model.add(Dropout(0.50))
    
    # Add two 3x3 convolution layers (output depth 64, and 64)
    model.add(Convolution2D(64, 3, 3, border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, border_mode='valid', W_regularizer=l2(0.001)))
    model.add(ELU())

    # Add a flatten layer
    model.add(Flatten())

    # Add three fully connected layers (depth 100, 50, 10), tanh activation (and dropouts)
    model.add(Dense(100, W_regularizer=l2(0.001)))
    model.add(ELU())
    #model.add(Dropout(0.50))
    model.add(Dense(50, W_regularizer=l2(0.001)))
    model.add(ELU())
    #model.add(Dropout(0.50))
    model.add(Dense(10, W_regularizer=l2(0.001)))
    model.add(ELU())
    #model.add(Dropout(0.50))

    # Add a fully connected output layer
    model.add(Dense(1))

    # Compile and train the model, 
    #model.compile('adam', 'mean_squared_error')
    model.compile(optimizer=Adam(lr=1e-4), loss='mse',metrics=['accuracy'])

    return model

In [4]:
#https://github.com/rekon/CarND-Traffic-Light-Classifier/blob/master/train_and_save_model.py
#https://medium.com/@arindambaidya168/https-medium-com-arindambaidya168-using-keras-imagedatagenerator-b94a87cdefad

%cd /content/drive/'My Drive'/'All_Repos'/Self-Driving-Car-NanoDegree/CarND-Behavioral-Cloning-P3/
import os
import numpy as np
import pandas as pd
import random
import cv2
import csv
import glob
from sklearn import model_selection

from keras import backend as K
from keras import models, optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Cropping2D, Dense, Dropout, Flatten, Lambda, Activation, MaxPooling2D, Reshape
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

from sklearn.utils import shuffle
import pandas as pd

if __name__ == "__main__":
  MODEL_FILE_NAME = './model.h5'
  df_front = pd.read_csv('data/set_1/driving_log.csv',names=['center','left','right','steer_angle','throtle','brake','speed'])
  df_reverse = pd.read_csv('data/set_2/driving_log.csv',names=['center','left','right','steer_angle','throtle','brake','speed'])
  df = df_front.append([df_reverse])

  # Split data into random training and validation sets
  d_train, d_valid = model_selection.train_test_split(df, test_size=.2,shuffle=True)

  train_gen = generator(d_train)
  validation_gen = generator(d_valid)

  model = get_model()

  checkpoint = ModelCheckpoint(MODEL_FILE_NAME, monitor='val_loss', verbose=1,save_best_only=True, mode='min',save_weights_only=False)
  callbacks_list = [checkpoint] #,callback_each_epoch]
  print('Training started....')
    
  history = model.fit_generator(train_gen,
                                steps_per_epoch=len(d_train)//BATCH_SIZE,
                                epochs=EPOCHS,
                                validation_data=validation_gen,
                                validation_steps=len(d_valid)//BATCH_SIZE,
                                verbose=1,
                                callbacks=callbacks_list)
  
  K.clear_session()

/content/drive/My Drive/All_Repos/Self-Driving-Car-NanoDegree/CarND-Behavioral-Cloning-P3


Using TensorFlow backend.
W0716 19:28:36.943856 139901461837696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0716 19:28:36.958105 139901461837696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), padding="valid", kernel_regularizer=<keras.reg...)`
  
W0716 19:28:36.967371 139901461837696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/usr/local/lib/python3.6/dist-packages/ipyke

Training started....


W0716 19:28:37.371927 139901461837696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0716 19:28:37.577774 139901461837696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/5
101/101 [==============================] - 141s 1s/step - loss: 0.4867 - acc: 0.0000e+00 - val_loss: 0.4636 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.46361, saving model to ./model.h5
Epoch 2/5
101/101 [==============================] - 127s 1s/step - loss: 0.4511 - acc: 0.0000e+00 - val_loss: 0.4442 - val_acc: 1.0000

Epoch 00002: val_loss improved from 0.46361 to 0.44420, saving model to ./model.h5
Epoch 3/5
 14/101 [===>..........................] - ETA: 38s - loss: 0.4393 - acc: 0.0000e+00

KeyboardInterrupt: ignored

In [0]:
# Path Join using a list needs '*' -  https://stackoverflow.com/questions/14826888/python-os-path-join-on-a-list
print('data/'+os.path.join(*d_train.center.iloc[0].replace('\\', '/').split('/')[-3:]))